In [ ]:
""" 
Python version of UK Government speeches web scraping exercise  
Christopher Gandrud (with ChatGPT 4)
2023-05-13
"""


import requests
from bs4 import BeautifulSoup
import pandas as pd

all_href = []
i = 0
while True:
    i += 1
    url = f"https://www.gov.uk/search/news-and-communications?level_one_taxon=5b7b9532-a775-4bd2-a3aa-6ce380184b6c&order=updated-newest&page={i}&people%5B%5D=boris-johnson"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tmp_href = [link.get('href') for link in soup.select('#js-results div ul li a')]
    if len(tmp_href) == 0:
        break
    all_href += tmp_href

all_texts = pd.DataFrame()
for href in all_href:
    url_one = f"https://www.gov.uk{href}"
    response = requests.get(url_one)
    soup = BeautifulSoup(response.text, 'html.parser')
    text = soup.select_one('#content div:nth-of-type(3) div:nth-of-type(1) div div:nth-of-type(1) div[data-module="govspeak"]').text
    tmp_df = pd.DataFrame({"url": [href], "text": [text]})
    all_texts = pd.concat([all_texts, tmp_df], ignore_index=True)

print(all_texts.describe())


This is the equivalent R code:

```r
xfun::pkg_attach2("roperators", "tidyverse", "rvest") # use infix operator %+=%

all_href <- character() # initialise all results object
tmp_href <- 1 # initalise temp object
i <- 0 # initalise counter
while (length(tmp_href) > 0 ) {
  i %+=% 1
  url <- paste0("https://www.gov.uk/search/news-and-communications?",
                "level_one_taxon=5b7b9532-a775-4bd2-a3aa-6ce380184b6c&",
                "order=updated-newest&page=", 
                as.character(i), 
                "&people%5B%5D=boris-johnson")
  tmp_href <- read_html(url) %>%
              html_elements(xpath = '//*[@id="js-results"]/div/ul/li[*]/a') %>%
                         html_attr('href')
  all_href <- c(all_href, tmp_href)
}

all_texts <- data.frame()
for (i in seq(all_href)) {
  url_one <- paste0("https://www.gov.uk", all_href[i])
  # message(url_one)
  full_html <- read_html(url_one)
  text <-  full_html %>% 
    html_nodes(xpath = 
      paste0('//*[@id="content"]/div[3]/div[1]/div',
             '/div[1]/div[contains(@data-module, "govspeak")]')) %>% 
    html_text()
  tmp_df <- data.frame(url = all_href[i], text = text)
  all_texts <- bind_rows(all_texts, tmp_df)
}
announce_corpus <- corpus(all_texts)
summary(announce_corpus)
```